In [1]:
import pandas as pd
import datetime as dt
import requests
import time
import numpy as np
import emoji

# Kaggle

In [2]:
df = pd.read_csv('Tweets.csv')
print(df.shape)
df.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df.values[0]

for i in range(len(df.columns)):
    print(df.columns[i], ' : ', df.values[3][i])

tweet_id  :  570301031407624196
airline_sentiment  :  negative
airline_sentiment_confidence  :  1.0
negativereason  :  Bad Flight
negativereason_confidence  :  0.7033
airline  :  Virgin America
airline_sentiment_gold  :  nan
name  :  jnardino
negativereason_gold  :  nan
retweet_count  :  0
text  :  @VirginAmerica it's really aggressive to blast obnoxious "entertainment" in your guests' faces &amp; they have little recourse
tweet_coord  :  nan
tweet_created  :  2015-02-24 11:15:36 -0800
tweet_location  :  nan
user_timezone  :  Pacific Time (US & Canada)


In [4]:
df.shape

(14640, 15)

In [5]:
# let's drop unwanted columns
df = df.drop(['tweet_id', 'airline_sentiment_confidence', 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 
'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_location', 'user_timezone'], axis=1)
df.head(1)

,airline_sentiment,airline,name,text,tweet_created
0,neutral,Virgin America,cairdin,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800


In [6]:
df["date"] = pd.to_datetime(df["tweet_created"])
df["date"] = df["date"].dt.strftime('%Y-%m-%d')
df["date"].head(), df["date"].max(), df["date"].min()

(0    2015-02-24
 1    2015-02-24
 2    2015-02-24
 3    2015-02-24
 4    2015-02-24
 Name: date, dtype: object,
 '2015-02-24',
 '2015-02-16')

In [7]:
df = df.drop(['tweet_created'], axis=1)
df.head(1)

,airline_sentiment,airline,name,text,date
0,neutral,Virgin America,cairdin,@VirginAmerica What @dhepburn said.,2015-02-24


In [8]:
unique, counts = np.unique(df['airline'], return_counts=True)

dict(zip(unique, counts))

{'American': 2759,
 'Delta': 2222,
 'Southwest': 2420,
 'US Airways': 2913,
 'United': 3822,
 'Virgin America': 504}

In [9]:
df = df.drop(df[(df['airline'] == 'US Airways') | (df['airline'] == 'Virgin America')].index)
unique, counts = np.unique(df['airline'], return_counts=True)

dict(zip(unique, counts))

{'American': 2759, 'Delta': 2222, 'Southwest': 2420, 'United': 3822}

In [10]:
df['name'].value_counts().head(10)

JetBlueNews       62
kbosspotter       32
_mhertz           29
otisday           28
throthra          27
scoobydoo9749     21
flemmingerin      19
ThatJasonEaton    18
Aero0729          18
chagaga2013       18
Name: name, dtype: int64

In [11]:
df.shape

(11223, 5)

In [12]:
df.to_csv('tweets_formatted.csv')  

# Reddit

In [13]:
df = pd.read_csv('americanairlines.csv')
print(df.shape)
df.head()

(9000, 122)


c:\Users\gakoo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (8,14,17,22,23,25,26,29,31,32,42,45,48,49,50,51,54,55,56,57,58,60,61,63,67,71,72,73,77,84,88,90,91,97,107,111,112,113,115,116) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,subreddit,selftext,author_fullname,gilded,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,...,saved,selftext_html,ups,user_reports,brand_safe,approved_at_utc,banned_at_utc,from_kind,from,from_id
0,0,americanairlines,My husband and I have 5 kids. We started the t...,t2_7zlrw6ih,0,Anyone enjoy figuring out best points scheme f...,[],r/americanairlines,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,americanairlines,"I’m new to the status thing, and I just reache...",t2_52b5349m,0,Main cabin Vs basic economy for upgrades,[],r/americanairlines,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,americanairlines,NaN,t2_dbp8phj,0,Domestic First meal comparison: Same menu item...,[],r/americanairlines,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,americanairlines,The site lets corporate travelers select seats...,t2_tqegvg14,0,Seatselect.AA.com is gone?,[],r/americanairlines,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,americanairlines,Basic Main Cabin Business ...,t2_6b0mnmgjy,0,I wonder which Flagship Transcend Option I'm C...,[],r/americanairlines,False,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
for i in range(len(df.columns)):
    print(df.columns[i], ' : ', df.values[3][i])

Unnamed: 0  :  3
subreddit  :  americanairlines
selftext  :  The site lets corporate travelers select seats. It shows a 404 today. Anyone else experience this? Is there a new site to replace it?
author_fullname  :  t2_tqegvg14
gilded  :  0
title  :  Seatselect.AA.com is gone?
link_flair_richtext  :  []
subreddit_name_prefixed  :  r/americanairlines
hidden  :  False
pwls  :  6.0
link_flair_css_class  :  nan
thumbnail_height  :  nan
top_awarded_type  :  nan
hide_score  :  True
quarantine  :  False
link_flair_text_color  :  dark
upvote_ratio  :  1.0
author_flair_background_color  :  nan
subreddit_type  :  public
total_awards_received  :  0.0
media_embed  :  {}
thumbnail_width  :  nan
author_flair_template_id  :  nan
is_original_content  :  False
secure_media  :  nan
is_reddit_media_domain  :  False
is_meta  :  False
category  :  nan
secure_media_embed  :  {}
link_flair_text  :  Question
score  :  1
is_created_from_ads_ui  :  False
author_premium  :  False
thumbnail  :  self
edited  :  Fal

In [15]:
columns = ['subreddit', 'selftext', 'title', 'author', 'utc_datetime_str']
for i in range(len(columns)):
    print(columns[i], ' : ', df[columns[i]].values[15])

subreddit  :  americanairlines
selftext  :  Do you find it annoying when people block the boarding area during boarding.  They announce the group numbers they are currently boarding but these idiots that have later boarding groups have to stand by the gate making it awkward AF if you need to navigate past them.  Just sit the F down until your group is called.
title  :  Blocking the boarding area
author  :  vw2xb
utc_datetime_str  :  2023-04-21 23:48:01


In [16]:
# remove features that are not in the list columns
df = df.drop([col for col in df.columns if col not in columns], axis=1)
df.head(1)

,subreddit,selftext,title,author,utc_datetime_str
0,americanairlines,My husband and I have 5 kids. We started the t...,Anyone enjoy figuring out best points scheme f...,blackjellybeansrule,2023-04-22 20:58:17


In [17]:
# merge selftext and title even if selftext is empty
df['text'] = df['title'].fillna('') + ' ' + df['selftext'].fillna('')
df = df.drop(['selftext', 'title'], axis=1)
df.head(1)

,subreddit,author,utc_datetime_str,text
0,americanairlines,blackjellybeansrule,2023-04-22 20:58:17,Anyone enjoy figuring out best points scheme f...


In [18]:
df.shape

(9000, 4)

In [19]:
df["date"] = pd.to_datetime(df["utc_datetime_str"])
df["date"] = df["date"].dt.strftime('%Y-%m-%d')
df = df.drop(['utc_datetime_str'], axis=1)

df["date"].head(1), df["date"].max(), df["date"].min()

(0    2023-04-22
 Name: date, dtype: object,
 '2023-04-22',
 '2016-05-20')

In [20]:
# rename columns name
df = df.rename(columns={'subreddit': 'airline', 'author': 'name'})
df.head(1)

,airline,name,text,date
0,americanairlines,blackjellybeansrule,Anyone enjoy figuring out best points scheme f...,2023-04-22


In [21]:
df['name'].value_counts().head(10)

[deleted]               1309
cocallaw                  45
JonathanDP81              38
ConsistentDriver9520      36
palkiajack                25
aboland96                 23
18809867070               21
My_fight                  21
Final-Craft-6992          19
cjasonac                  18
Name: name, dtype: int64

In [22]:
df[df['name'] == '[deleted]'].head(10)

,airline,name,text,date
720,americanairlines,[deleted],"Hey y’all, I just logged in and I’m having a H...",2023-03-01
735,americanairlines,[deleted],I’ve flown.. a year that’s full… and each and ...,2023-03-01
739,americanairlines,[deleted],JAL lounge @HND terminal 3 [deleted],2023-03-01
1015,americanairlines,[deleted],What’s the Muslim meal on American Airlines li...,2023-02-11
1595,americanairlines,[deleted],Any website where you can tell *why* a plane w...,2022-12-28
1637,americanairlines,[deleted],Does status cover next year too? [deleted],2022-12-24
1641,americanairlines,[deleted],Separated on a 10+ hour flight [deleted],2022-12-24
1646,americanairlines,[deleted],Not seeing my family for Christmas [deleted],2022-12-24
1655,americanairlines,[deleted],OKC to NRT - Need to get separate ticket after...,2022-12-22
1671,americanairlines,[deleted],Can I fly with two infants on AA? [deleted],2022-12-22


# now combine all the dataframes into one

In [23]:
all_dfs = []
csv_names = ['americanairlines.csv', 'delta.csv', 'united.csv'] # add southwest.csv
for csv_name in csv_names:
    df = pd.read_csv(csv_name)
    columns = ['subreddit', 'selftext', 'title', 'author', 'utc_datetime_str']
    df = df.drop([col for col in df.columns if col not in columns], axis=1)
    df['text'] = df['title'].fillna('') + ' ' + df['selftext'].fillna('')
    df = df.drop(['selftext', 'title'], axis=1)
    df["date"] = pd.to_datetime(df["utc_datetime_str"])
    df["date"] = df["date"].dt.strftime('%Y-%m-%d')
    df = df.drop(['utc_datetime_str'], axis=1)
    df = df.rename(columns={'subreddit': 'airline', 'author': 'name'})
    all_dfs.append(df)
    print('='*50, csv_name, '='*50)
    print('shape :', df.shape)
    print('top 3 account :', df['name'].value_counts().head(3))
    print('first post :', df["date"].min())
    print('last post :', df["date"].max())


print('='*50, 'final csv', '='*50)
df = pd.concat(all_dfs)
#df['text'] = df['text'].apply(lambda s: emoji.replace_emoji(s, ''))
print('final shape :', df.shape)
print('first post :', df["date"].min())
print('last post :', df["date"].max())
print(df.head())

df.to_csv('reddit.csv')

c:\Users\gakoo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (8,14,17,22,23,25,26,29,31,32,42,45,48,49,50,51,54,55,56,57,58,60,61,63,67,71,72,73,77,84,88,90,91,97,107,111,112,113,115,116) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


================================================== americanairlines.csv ==================================================
shape : (9000, 4)
top 3 account : [deleted]       1309
cocallaw          45
JonathanDP81      38
Name: name, dtype: int64
first post : 2016-05-20
last post : 2023-04-22


c:\Users\gakoo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (46,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


================================================== delta.csv ==================================================
shape : (9029, 4)
top 3 account : [deleted]    506
A350Flier     32
Idealmonk     26
Name: name, dtype: int64
first post : 2022-07-07
last post : 2023-04-22


c:\Users\gakoo\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (14,23,26,31,32,42,45,46,48,49,54,55,57,61,67,71,89,91,93,97,107,111,112,113,115,116,117) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


================================================== united.csv ==================================================
shape : (9000, 4)
top 3 account : [deleted]       1142
doc_ocho          66
Seniorhusky1      37
Name: name, dtype: int64
first post : 2017-12-06
last post : 2023-04-21
================================================== final csv ==================================================
final shape : (27029, 4)
first post : 2016-05-20
last post : 2023-04-22
            airline                 name  \
0  americanairlines  blackjellybeansrule   
1  americanairlines         CriminalBoss   
2  americanairlines    LadyEmeraldDeVere   
3  americanairlines       essentially_no   
4  americanairlines             tyster30   

                                                text        date  
0  Anyone enjoy figuring out best points scheme f...  2023-04-22  
1  Main cabin Vs basic economy for upgrades I’m n...  2023-04-22  
2  Domestic First meal comparison: Same menu item...  2023-04-22  
3